<a href="https://colab.research.google.com/github/CalculatedContent/WeightWatcher-Examples/blob/main/WW_Mistral_DragonKings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install weightwatcher
import weightwatcher as ww
ww.__version__

In [2]:
base_model = "Mistral-7B-v0.1"
base_model_html = "https://huggingface.co/mistralai/Mistral-7B-v0.1"

!git clone $base_model_html

Cloning into 'Mistral-7B-v0.1'...
remote: Enumerating objects: 87, done.
remote: Total 87 (delta 0), reused 0 (delta 0), pack-reused 87
Unpacking objects: 100% (87/87), 474.17 KiB | 2.03 MiB/s, done.
Filtering content: 100% (5/5), 3.46 GiB | 2.23 MiB/s, done.
Encountered 4 file(s) that may not have been copied correctly on Windows:
	model-00002-of-00002.safetensors
	pytorch_model-00002-of-00002.bin
	model-00001-of-00002.safetensors
	pytorch_model-00001-of-00002.bin

See: `git lfs help smudge` for more details.


In [3]:
!ls $base_model

config.json			  pytorch_model-00001-of-00002.bin  tokenizer_config.json
generation_config.json		  pytorch_model-00002-of-00002.bin  tokenizer.json
model-00001-of-00002.safetensors  pytorch_model.bin.index.json	    tokenizer.model
model-00002-of-00002.safetensors  README.md
model.safetensors.index.json	  special_tokens_map.json


### Sanity check: run Describe

In [4]:
import logging, os
watcher = ww.WeightWatcher(log_level=logging.WARNING)
details = watcher.describe(model=base_model)

In [ ]:
details

### Run Analyze, get a details dataframe with layer quality metrics

In [ ]:
import logging, os
watcher = ww.WeightWatcher(log_level=logging.WARNING)
details = watcher.analyze(model=base_model, fix_fingers='clip_xmax', randomize=True)

In [ ]:
import matplotlib.pyplot as plt
D = details

ax = D.alpha.plot.hist(bins=100, title=f"WeightWatcher Quality Metrics for {base_model}", label="alpha fixed")
D.raw_alpha.plot.hist(bins=100, label="alpha raw", alpha=0.5, color='purple')

avg_alpha = D.alpha.mean()
plt.axvline(x=avg_alpha, color='blue',label=f"<alpha> = {avg_alpha:0.3f}")

avg_raw_alpha = D.raw_alpha.mean()
plt.axvline(x=avg_raw_alpha, color='purple',label=f"<raw alpha> = {avg_raw_alpha:0.3f}")


plt.axvline(x=2.0, color='red',label=f"overfit warning")

plt.xlabel("WeightWatcher Quality Metric Alpha")
plt.legend()
filename = f"{base_model_dir}/alphas.png"
plt.savefig(filename)

### Plot the ESD of a single layer with a finger

In [ ]:
watcher.analyze(layers=[139], plot=True)

### Plot alpha and raw_alpha vs rand_distance

In [ ]:
D.plot.scatter(x='alpha', y='rand_distance')
D.plot.scatter(x='raw_alpha', y='rand_distance')